# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [193]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [5]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [6]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [7]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [8]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [9]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [10]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [11]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [12]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [112]:
grouped = data.groupby(["ProductName", "CustomerID"]).agg({"Quantity": "sum"})
pivot = pd.pivot_table(grouped, values="Quantity", index="CustomerID", columns="ProductName", fill_value=0, dropna=False)
pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [113]:
dist = pdist(pivot.T, metric="euclidean")
dist

array([279.99642855, 253.66907577, 238.5120542 , ..., 264.14200726,
       268.28343221, 273.15014186])

In [114]:
sqForm = squareform(dist)
sqForm

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

In [115]:
newDf = pd.DataFrame(1/(1 + sqForm), index=pivot.columns, columns=pivot.columns)
newDf.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.003559,0.003927,0.004175,0.004102,0.003616,0.003767,0.003876,0.004024,0.004023,...,0.003794,0.003749,0.003846,0.004037,0.003839,0.004026,0.003908,0.003896,0.003860,0.003578
"Appetizer - Mini Egg Roll, Shrimp",0.003559,1.000000,0.003670,0.003697,0.003692,0.003413,0.003479,0.003900,0.003789,0.003486,...,0.003639,0.003481,0.003624,0.003751,0.003639,0.003508,0.003691,0.003749,0.003564,0.003475
Appetizer - Mushroom Tart,0.003927,0.003670,1.000000,0.004234,0.003849,0.003789,0.003819,0.004120,0.004110,0.003686,...,0.003920,0.003683,0.003812,0.004418,0.004155,0.004111,0.004095,0.004060,0.003937,0.003698
Appetizer - Sausage Rolls,0.004175,0.003697,0.004234,1.000000,0.004078,0.003974,0.003808,0.004169,0.003998,0.003837,...,0.003922,0.003853,0.003962,0.004551,0.004089,0.003872,0.004082,0.004020,0.003878,0.003718
Apricots - Dried,0.004102,0.003692,0.003849,0.004078,1.000000,0.003755,0.003722,0.004073,0.003895,0.003778,...,0.003760,0.003669,0.003869,0.004233,0.003836,0.003718,0.004018,0.003955,0.003811,0.003767


### Step 2: Get the products purchased for a specific customer of your choice.

In [124]:
cust33 = pivot.loc[33][pivot.loc[33]>0]
cust33 = cust33.sort_values(ascending=False)
print(len(cust33))
cust33 = cust33.index.tolist()
print(cust33)

59
['Lettuce - Spring Mix', 'Grouper - Fresh', 'Veal - Inside, Choice', 'Salsify, Organic', 'Yogurt - French Vanilla', 'Cassis', 'Lamb - Pieces, Diced', 'Ice Cream Bar - Drumstick', 'Hersey Shakes', 'Grapes - Red', 'General Purpose Trigger', 'French Pastry - Mini Chocolate', 'Fondant - Icing', 'Crab - Imitation Flakes', 'Cheese - Cottage Cheese', 'Cake - Box Window 10x10x2.5', 'Muffin Batt - Choc Chk', 'Bread Foccacia Whole', 'Bread - Italian Roll With Herbs', 'Blackberries', 'Black Currants', 'Beer - Rickards Red', 'Beer - Original Organic Lager', 'Beef - Prime Rib Aaa', 'Beef - Chuck, Boneless', 'Beans - Kidney, Canned', 'Bar Mix - Pina Colada, 355 Ml', 'Bandage - Flexible Neon', 'Assorted Desserts', 'Mangoes', 'Pasta - Cheese / Spinach Bauletti', 'Onions - Vidalia', 'Wine - Valpolicella Masi', 'Wine - Two Oceans Cabernet', 'Wine - Sogrape Mateus Rose', 'Wine - Gato Negro Cabernet', 'Wine - Crozes Hermitage E.', 'Water - Green Tea Refresher', 'Veal - Sweetbread', 'Veal - Eye Of Round

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [126]:
itemList = []
for item in cust33:
    itemList.append(newDf[item].sort_values(ascending=False)[1:].nlargest(5).index.tolist())
itemList
flatItems = [item for itemItem in itemList for item in itemItem]
toPurchase = [item for item in flatItems if item not in cust33]
print(len(toPurchase))

248


### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [155]:
numProd = {}
for item in toPurchase:
    numProd[item] = toPurchase.count(item)
ans = [(item, numProd[item]) for item in numProd]
ans.sort(key= lambda x: x[1], reverse=True)
answ_val = [(a[0], a[1]) for a in ans][:5]
answ = [a[0] for a in ans][:5]
answ

['Rabbit - Whole',
 'Scallops 60/80 Iqf',
 'Bar - Granola Trail Mix Fruit Nut',
 'Wine - Wyndham Estate Bin 777',
 'Steam Pan - Half Size Deep']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [167]:
customers = list(set(data["CustomerID"]))
recomendation = {}
for custom in customers:
    cust = pivot.loc[custom][pivot.loc[custom]>0]
    cust = cust.sort_values(ascending=False)
    cust = cust.index.tolist()
    itemList = []
    for item in cust:
        itemList.append(newDf[item].sort_values(ascending=False)[1:].nlargest(5).index.tolist())
    itemList
    flatItems = [item for itemItem in itemList for item in itemItem]
    toPurchase = [item for item in flatItems if item not in cust]
    numProd = {}
    for item in toPurchase:
        numProd[item] = toPurchase.count(item)
    ans = [(item, numProd[item]) for item in numProd]
    ans.sort(key= lambda x: x[1], reverse=True)
    answ_val = [(a[0], a[1]) for a in ans][:5]
    answ = [a[0] for a in ans][:5]
    recomendation[f"{custom}"] = answ

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [181]:
recoDf = pd.DataFrame.from_dict(recomendation)
recoDfT = recoDf.T
recoDfT.columns = ["Item 1", "Item 2", "Item 3", "Item 4", "Item 5"]
recoDfT

,Item 1,Item 2,Item 3,Item 4,Item 5
83973,Rabbit - Whole,Water - Green Tea Refresher,Scallops 60/80 Iqf,Steam Pan - Half Size Deep,Bar - Granola Trail Mix Fruit Nut
59399,Rabbit - Whole,Water - Green Tea Refresher,"Pork - Loin, Center Cut",Bar - Granola Trail Mix Fruit Nut,"Juice - Cranberry, 341 Ml"
92168,Rabbit - Whole,Water - Green Tea Refresher,Scallops 60/80 Iqf,"Pork - Loin, Center Cut",Raspberries - Fresh
49159,Water - Green Tea Refresher,Rabbit - Whole,Scallops 60/80 Iqf,Bar - Granola Trail Mix Fruit Nut,"Pork - Loin, Center Cut"
18441,Water - Green Tea Refresher,"Pork - Loin, Center Cut","Juice - Cranberry, 341 Ml",Bar - Granola Trail Mix Fruit Nut,Bread - Hot Dog Buns
...,...,...,...,...,...
77814,Scallops 60/80 Iqf,Water - Green Tea Refresher,Rabbit - Whole,"Pork - Loin, Center Cut","Juice - Cranberry, 341 Ml"
32753,Water - Green Tea Refresher,Scallops 60/80 Iqf,Sauce - Hollandaise,Bar - Granola Trail Mix Fruit Nut,"Pork - Loin, Center Cut"
20476,Rabbit - Whole,Water - Green Tea Refresher,Scallops 60/80 Iqf,Bar - Granola Trail Mix Fruit Nut,"Pork - Loin, Center Cut"
67582,Rabbit - Whole,Water - Green Tea Refresher,Scallops 60/80 Iqf,"Juice - Cranberry, 341 Ml",Vol Au Vents


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [182]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [204]:
prod_cust_pivot[70000] = pd.DataFrame.from_dict(new_customer, orient="index")
prod_cust_pivot = prod_cust_pivot.fillna(0)
cust_prod_pivot = prod_cust_pivot.T
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)
similar_cust = list(cust_dist[70000].sort_values(ascending=False)[1:].head().index)
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[70000]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs.ProductName.tolist()

['Scallops - 10/20',
 'Browning Caramel Glace',
 'Beef - Top Sirloin - Aaa',
 'Skirt - 29 Foot',
 'Onions - Cippolini']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [205]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
products_cust = cust_prod_pivot.loc[70000][cust_prod_pivot.loc[70000] > 0]
similar_prod = [list(prod_dist[e].sort_values(ascending=False)[1:].head().index) for e in products_cust.index]
similar_prod = [j for e in similar_prod for j in e]
sim_prod_cust = [product for product in similar_prod if product not in products_cust.index]
number_products = {e:sim_prod_cust.count(e) for e in sim_prod_cust}
number_products = sorted(number_products.items(), key=lambda x: x[1], reverse=True)
[e[0] for e in number_products][:5]

['Beef - Chuck, Boneless',
 'Towels - Paper / Kraft',
 'Rice - Jasmine Sented',
 'Initation Crab Meat',
 'V8 - Berry Blend']